# Segmantic Segmentation with VGG16

The initial attempt with using a GAN hit a wall, specifically with figuring out how to do proper layer sizing for the transposed convolution layers for the generator. On top of that, some research I did during my "off" time showed that the approach semantic segmentation using either an FCN (Fully Convolutional Network) approach or SegNet would work best.

Since SegNet is completely new, whereas transfer learning with VGG16 is something I'm familiar with, I'm going to attempt a VGG16 network here.

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from random import randint
from glob import glob
import os
import os.path

%matplotlib inline

import tensorflow as tf
#Check GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
print('TensorFlow Version: {}'.format(tf.__version__))

Default GPU Device: /device:GPU:0
TensorFlow Version: 1.6.0


In [3]:
#useful variables
num_classes = 13 # none and 12 options, 0-12
image_shape = (160, 576)
weights_initializer_stddev = 0.01
weights_regularized_l2 = 1e-3

## Load VGG16


First we're going to load VGG16 with pretrained weights (so it maintains its feature detectors, which can be useful for our smaller dataset).

In [4]:
#Download VGG16 if it is not already
from urllib.request import urlretrieve
import zipfile

if not os.path.exists("vgg16.zip"):
    urlretrieve(
        'https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/vgg.zip',
        "./vgg16.zip")
    print("Downloaded VGG16 model weights")
else:
    print("Already exists, skipping download")

Already exists, skipping download


In [5]:
#Extract if needed
if not os.path.exists("./vgg"):
    unzip = zipfile.ZipFile("./vgg16.zip", "r")
    unzip.extractall("./")
    unzip.close()
    print("Extracted VGG16 model weights")
else:
    print("Already exists, skipping extraction")

Already exists, skipping extraction


In [6]:
#Make a map more colorful
def convertToColor(value):
    colors = [
        (255, 255, 255),   #0
        (255, 0, 0),      #1
        (0, 255, 0),      #2
        (0, 0, 255),      #3
        (255, 255, 0),    #4
        (127, 0, 255),    #5
        (51, 255, 51),    #6
        (255, 0, 127),    #7
        (127, 127, 127),  #8
        (0, 0, 0),        #9
        (0, 255, 255),  #10
        (0, 0, 100),      #11
        (100, 0, 0),      #12
    ]
    return colors[value[0]]

def colorizeMap(img):
    return [list( map(convertToColor, row) ) for row in img]

In [7]:
#We need to be able to convert the image map to a 13 channel ground truth map, and vice versa
def pixelToTruth(value):
    truths = [
        (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
        (0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
        (0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
        (0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0),
        (0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0),
        (0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0),
        (0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0),
        (0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0),
        (0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0),
        (0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0),
        (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0),
        (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0),
        (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1)
    ]
    
    return truths[value[0]]

def truthToPixel(value):
    return (value.tolist().index(1), 0, 0)

def imageToTruth(img):
    return [list(map(pixelToTruth, row)) for row in img]

def truthToImage(truth):
    return [list(map(truthToPixel, row)) for row in truth]

In [22]:
# Function to load the image data and the label for it
def get_training_data(batch_size):
    #Both inputs and ground truth maps have the same name - easy!
    image_paths = glob(os.path.join("./data/Train/CameraRGB", "*.png"))
    label_paths = glob(os.path.join("./data/Train/CameraSeg", "*.png"))
    
    chosen_image = '165.png';
    
    for batch in range(0, len(image_paths), batch_size):
        images = []
        maps = []
        
#         for image_file in image_paths[batch:batch + batch_size]:
#             image = cv2.imread("./data/Train/CameraRGB/{}".format(chosen_image))
#             image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#             image = cv2.resize(image, image_shape)
#             map_image = cv2.imread("./data/Train/CameraSeg/{}".format(chosen_image))
#             map_image = cv2.cvtColor(map_image, cv2.COLOR_BGR2RGB)
#             map_image = cv2.resize(map_image, image_shape)
#             map_image = imageToTruth(map_image)
            
#             images.append(image)
#             maps.append(map_image)
            
#         yield np.array(images), np.array(maps)
    
    for batch in range(0, len(image_paths), batch_size):
        images = []
        maps = []
        
        for index, image_file in enumerate(image_paths[batch:batch + batch_size]):
            map_file = os.path.join(label_paths[index])
            
            image = cv2.imread(image_file)
#             image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, image_shape)
#             image = image - np.array([123.68, 116.779, 103.939], dtype=np.float32)
            map_image = cv2.imread(map_file)
            map_image = cv2.cvtColor(map_image, cv2.COLOR_BGR2RGB)
            map_image = cv2.resize(map_image, image_shape)
            map_image = imageToTruth(map_image)
            
            images.append(image)
            maps.append(map_image)

        yield np.array(images), np.array(maps)
            
            

In [25]:
tf.reset_default_graph()
with tf.Session() as sess:
    #Placeholders
    label = tf.placeholder(tf.int32, (None, None, None, num_classes), name='label')
    learning_rate = tf.placeholder(tf.int32, name='learning_rate')
    
    #Grab layers from pretrained VGG
    tf.saved_model.loader.load(sess, ["vgg16"], "./vgg/")
    
    graph = tf.get_default_graph()
    
    #define key layers for us to work with, so we can take pieces of VGG16
    #for our own use
    input_layer = graph.get_tensor_by_name("image_input:0")
    keep_prob = graph.get_tensor_by_name("keep_prob:0") #Dropout settings
    
    #more layer grabbing
    layer_3 = graph.get_tensor_by_name("layer3_out:0")
    layer_4 = graph.get_tensor_by_name("layer4_out:0")
    layer_7 = graph.get_tensor_by_name("layer7_out:0")
    
#     kernel_initializer = tf.random_normal_initializer(stddev=weights_initializer_stddev),
#     kernel_regularizer= tf.contrib.layers.l2_regularizer(weights_regularized_l2)


    # Skip connections for later
    skip_conv_3 = tf.layers.conv2d(layer_3, num_classes, 1, padding='same',
            kernel_initializer = tf.random_normal_initializer(stddev=weights_initializer_stddev),
            kernel_regularizer= tf.contrib.layers.l2_regularizer(weights_regularized_l2))
    
    skip_conv_4 = tf.layers.conv2d(layer_4, num_classes, 1, padding='same',
            kernel_initializer = tf.random_normal_initializer(stddev=weights_initializer_stddev),
            kernel_regularizer= tf.contrib.layers.l2_regularizer(weights_regularized_l2))
    
    #Layer 7 isn't skipped, it's passed right to transpose    
    fully_connected_convs = tf.layers.conv2d(layer_7, num_classes, 1, padding='same',
            kernel_initializer = tf.random_normal_initializer(stddev=weights_initializer_stddev),
            kernel_regularizer= tf.contrib.layers.l2_regularizer(weights_regularized_l2))
    
    #From layer 7 we need to transpose up
    transpose_1 = tf.layers.conv2d_transpose(fully_connected_convs, num_classes, 4, 2, padding='same',
            kernel_initializer = tf.random_normal_initializer(stddev=weights_initializer_stddev),
            kernel_regularizer= tf.contrib.layers.l2_regularizer(weights_regularized_l2))
    
    # Add the skip layer from layer 4
    skip_1 = tf.add(transpose_1, skip_conv_4)
    
    #Tranpose up from resultant layer
    transpose_2 = tf.layers.conv2d_transpose(skip_1, num_classes, 4, 2, padding='same',
            kernel_initializer = tf.random_normal_initializer(stddev=weights_initializer_stddev),
            kernel_regularizer= tf.contrib.layers.l2_regularizer(weights_regularized_l2))
    
    #Create skip layer from layer 3
    skip_2 = tf.add(skip_conv_3, transpose_2)
    
    #Final output layer
    output_layer = tf.layers.conv2d_transpose(skip_2, num_classes, 16, 8, padding='same',
            kernel_initializer = tf.random_normal_initializer(stddev=weights_initializer_stddev),
            kernel_regularizer= tf.contrib.layers.l2_regularizer(weights_regularized_l2),
            activation=tf.sigmoid)
    
    #layer_15 will be our transposed output!
    
    #Create the optimzer
    logits = tf.reshape(output_layer, (-1, num_classes))
    correct_label = tf.reshape(label, (-1, num_classes))
#     mean_absolute = tf.metrics.mean_absolute_error(tf.cast(logits, tf.float32), tf.cast(correct_label, tf.float32))
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=correct_label))
    optimizer= tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(cross_entropy_loss)
#     train_op = optimizer.minimize(mean_absolute)
    
    
    #and now, training!
    epochs = 1
    batch_size = 10
    keep_probability = 0.5
    learning_rate_alpha = 0.0001
    
    saver = tf.train.Saver()
    
    sess.run(tf.global_variables_initializer())
    
    print("Graph has been built - launching training")
    print("====== :-) ======")
    print()
    
    for epoch in range(epochs):
        print("Launching Epoch {}".format(epoch))
        loss_log = []
        batch_count = 0
        
        #get the images
        for image, truth in get_training_data(batch_size):
            batch_count += 1
            _, loss = sess.run(
                    [train_op, cross_entropy_loss],
                    feed_dict = {
                        input_layer: image,
                        label: truth,
                        keep_prob: keep_probability,
                        learning_rate: learning_rate_alpha
                    }
                )
            loss_log.append('{:3f}'.format(loss))
            if(batch_count % 10 == 0):
                print("Batch {} - loss of {}".format(batch_count, loss))
        print("Training for epoch finished - ", loss_log[-1])
        print()
    print("Training finished")
    

INFO:tensorflow:Restoring parameters from b'./vgg/variables/variables'
Graph has been built - launching training
====== :-) ======

Launching Epoch 0


KeyboardInterrupt: 